#Quickstart

Working with data

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import numpy as np
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir="run/training")

In [8]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=T.ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=T.ToTensor(),
)

In [9]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Creating Models

In [10]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
# class MultilayerPerceptron(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(28*28, 128),
#             nn.ReLU(),
#             nn.Linear(128, 128),
#             nn.ReLU(),
#             nn.Linear(128, 10)
#         )

    # def forward(self, x):
    #     x = self.flatten(x)
    #     logits = self.linear_relu_stack(x)
    #     return logits

class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32, kernel_size=3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.1, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.1, training=self.training)
        x = self.fc3(x)
        return x

# model = MultilayerPerceptron().to(device)
cnn = ConvolutionalNeuralNetwork().to(device)
# print(model)

Using mps device


Optimizing the Model Parameters

In [11]:
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer2 = torch.optim.SGD(cnn.parameters(), lr=4e-2, momentum=0.9)

In [12]:
def train(dataloader, model, loss_fn, optimizer, epochs):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        step = epochs * len(dataloader) + batch
        writer.add_scalar("Loss/train", loss.item(), step)

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def train2(dataloader, model, loss_fn, optimizer, epoch):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        step = epoch * len(dataloader) + batch
        writer.add_scalar("Loss/train", loss.item(), step)

        if batch % 100 == 0:
            loss_value, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss_value:>7f}  [{current:>5d}/{size:>5d}]")

In [13]:
accuracies = []

In [14]:
def test(dataloader, model, loss_fn, epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    accuracy = correct / size
    
    # Enregistrement dans TensorBoard
    writer.add_scalar("Accuracy/test", accuracy, epoch)
    writer.add_scalar("Loss/test", test_loss, epoch)
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    # Enregistrement dans la liste
    if accuracies is not None:
        accuracies.append(accuracy)


In [15]:
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train2(train_dataloader, cnn, loss_fn, optimizer2, epoch)
    test(test_dataloader, cnn, loss_fn, epoch)
    writer.close()
print("Done!")


Epoch 1
-------------------------------
loss: 2.313233  [   64/60000]
loss: 0.214970  [ 6464/60000]
loss: 0.171183  [12864/60000]
loss: 0.234930  [19264/60000]
loss: 0.029141  [25664/60000]
loss: 0.109126  [32064/60000]
loss: 0.041320  [38464/60000]
loss: 0.098778  [44864/60000]
loss: 0.299609  [51264/60000]
loss: 0.108417  [57664/60000]
Test Error: 
 Accuracy: 980600.0%, Avg loss: 0.063748 

Epoch 2
-------------------------------
loss: 0.026640  [   64/60000]
loss: 0.170810  [ 6464/60000]
loss: 0.056210  [12864/60000]
loss: 0.132803  [19264/60000]
loss: 0.017373  [25664/60000]
loss: 0.055614  [32064/60000]
loss: 0.061350  [38464/60000]
loss: 0.025562  [44864/60000]
loss: 0.208103  [51264/60000]
loss: 0.080911  [57664/60000]
Test Error: 
 Accuracy: 985800.0%, Avg loss: 0.042210 

Epoch 3
-------------------------------
loss: 0.005996  [   64/60000]
loss: 0.088572  [ 6464/60000]
loss: 0.012793  [12864/60000]
loss: 0.086840  [19264/60000]
loss: 0.010466  [25664/60000]
loss: 0.068130  [3

Saving Models

In [16]:
# torch.save(model.state_dict(), "model.pth")
torch.save(cnn.state_dict(), "model2.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading Models

In [17]:
cnn2 = ConvolutionalNeuralNetwork().to(device)
cnn2.load_state_dict(torch.load("model2.pth", weights_only=True))

<All keys matched successfully>

In [18]:
# classes = [
#     "T-shirt/top",
#     "Trouser",
#     "Pullover",
#     "Dress",
#     "Coat",
#     "Sandal",
#     "Shirt",
#     "Sneaker",
#     "Bag",
#     "Ankle boot",
# ]

# model.eval()
# x, y = test_data[0][0], test_data[0][1]
# with torch.no_grad():
#     x = x.to(device)
#     pred = model(x)
#     predicted, actual = classes[pred[0].argmax(0)], classes[y]
#     print(f'Predicted: "{predicted}", Actual: "{actual}"')

TensorBoard

In [19]:
import statistics
mean_acc = statistics.mean(accuracies)
median_acc = statistics.median(accuracies)

print(f"Moyenne des accuracies test sur {epochs} époques : {mean_acc:.4f}")
print(f"Médiane des accuracies test : {median_acc:.4f}")

writer.add_scalar("Stats/mean_accuracy", mean_acc, epochs)
writer.add_scalar("Stats/median_accuracy", median_acc, epochs)
writer.close()
print("Done!")

Moyenne des accuracies test sur 5 époques : 0.9863
Médiane des accuracies test : 0.9877
Done!


In [23]:
print(cnn)
cnn.eval()
print(cnn)
dummy_input = torch.randn(1, 1, 28, 28, device=device)
torch.onnx.export(
    cnn, 
    dummy_input, 
    "backend/mnist_cnn2.onnx",
    input_names=["input"], 
    output_names=["output"], 
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11
)
print("Modèle exporté en mnist_cnn.onnx")

ConvolutionalNeuralNetwork(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)
ConvolutionalNeuralNetwork(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)
Modèle exporté en mnist_cnn.onnx


In [21]:
from PIL import Image
import numpy as np
import onnxruntime as ort

img = Image.open("debug_digit.png").convert("L")
img = np.array(img).astype(np.float32) / 255.0
img = (img - 0.1307) / 0.3081
input_tensor = img.reshape(1, 1, 28, 28)

session = ort.InferenceSession("mnist_cnn.onnx")
outputs = session.run(None, {"input": input_tensor})
print("Prediction:", np.argmax(outputs[0]))


Prediction: 8
